# OJA Deduplication Challenge

## Imports

### Packages

In [58]:
import os
import warnings

import pandas as pd
from tqdm import tqdm

from Levenshtein import distance
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [59]:
warnings.filterwarnings('ignore')

### Data

In [60]:
# Import from s3
os.system(f"mc cp s3/apalazzolo/Deduplication/wi_dataset.csv wi_dataset.csv")

`s3/apalazzolo/Deduplication/wi_dataset.csv` -> `wi_dataset.csv`
Total: 0 B, Transferred: 203.23 MiB, Speed: 142.03 MiB/s


0

In [61]:
original_data = pd.read_csv('wi_dataset.csv',
                            lineterminator='\n')

In [62]:
# For now let's work with a smaller extract
data = original_data.head(10000)

In [63]:
n_ads = len(data)
n_ads

10000

In [64]:
data.head(3)

,id,title,description,location,country_id,company_name,retrieval_date
0,1,Traineeship Pensioen & Leven in Utrecht,We sturen je door naar je toekomstige opdracht...,Utrecht,NL,NaN,2021-01-19
1,2,DEPOSITARY OFFICER (M/F),DEPOSITARY OFFICER (M/F) DO Recruitment Adviso...,NaN,FR,DO Recruitment Advisors,2021-09-30
2,3,Cautam colegi manipulanti marfa,Descriere Angajam manipulanti marfa din Pitest...,Pitesti,RO,NaN,2021-06-18


## Data preprocessing

In [65]:
# Cleaning basique

data[
    ['title', 'description', 'location', 'company_name']
] = data[
    ['title', 'description', 'location', 'company_name']
].apply(lambda x: x.str.strip().str.lower())

data.fillna("", inplace=True)

## Naive deduplication

In [66]:
duplicates = []

### Add the full duplicates

In [67]:
# Beaucoup trop long
# Attention, si on trie la table rien ne marche plus !

# for i in tqdm(range(n_ads)):
#     for j in range(i+1,n_ads):
#         if (data.iloc[i, 1] == data.iloc[j, 1]) and (data.iloc[i, 2] == data.iloc[j, 2]):
#             duplicates.append({'id1': i+1, 'id2': j+1, 'type': 'FULL'})

In [68]:
data.sort_values(by=['title', 'description', 'id'], inplace=True)

i = 0
j = 1

while i < n_ads and j < n_ads:
    if (i%10000 == 0) and (j == i+1):
        print(i)
    if data.iloc[i, 1] < data.iloc[j, 1]:
        i += 1
        j = i + 1
    elif data.iloc[i, 1] > data.iloc[j, 1]:
        j += 1
    elif data.iloc[i, 2] < data.iloc[j, 2]:
        i += 1
        j = i + 1
    elif data.iloc[i, 2] > data.iloc[j, 2]:
        j += 1
    else:
        duplicates.append({'id1': data.iloc[i, 0], 'id2': data.iloc[j, 0], 'type': 'FULL'})
        j += 1

0


In [69]:
len(duplicates)

2753

In [70]:
duplicates[0]

{'id1': 236, 'id2': 1598, 'type': 'FULL'}

### Add the semantic duplicates

In [71]:
data.sort_values(by=['id'], inplace=True)

In [72]:
data['text'] = data['title'] + ' ' + data['description'] + ' ' + data['location'] + ' ' + data['company_name']

# Utiliser TF-IDF pour vectoriser les textes
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(data['text'])

In [73]:
# Utiliser la similarité cosinus pour comparer les textes
similarity_matrix = cosine_similarity(tfidf)

In [ ]:
# Identifier les paires d'annonces similaires

for i in tqdm(range(similarity_matrix.shape[0])):
    for j in range(i+1, similarity_matrix.shape[1]):
        if similarity_matrix[i][j] > 0.8 and (
            (data.iloc[i, 1] != data.iloc[j, 1]) or (data.iloc[i, 2] != data.iloc[j, 2])
        ):
            if data.iloc[i, 6] != data.iloc[j, 6]:
                duplicates.append({'id1': data.iloc[i, 0], 'id2': data.iloc[j, 0], 'type': 'TEMPORAL'})
            else:
                if abs(
                    len(data.iloc[i, 2]) - len(data.iloc[j, 2])
                ) / max(
                    len(data.iloc[i, 2]), len(data.iloc[j, 2])
                ) < 0.08:
                    duplicates.append({'id1': data.iloc[i, 0], 'id2': data.iloc[j, 0], 'type': 'SEMANTIC'})
                else:
                    duplicates.append({'id1': data.iloc[i, 0], 'id2': data.iloc[j, 0], 'type': 'PARTIAL'})

 26%|██▌       | 2580/10000 [00:07<00:18, 392.46it/s]

## Print duplicates

In [ ]:
duplicates = pd.DataFrame(duplicates)
duplicates.sort_values(by=['id1', 'id2'], inplace=True)
duplicates

In [ ]:
duplicates.groupby('type').count()

In [ ]:
duplicates.to_csv('duplicates.csv', index=False)

In [ ]:
os.system(f"mc cp duplicates.csv s3/apalazzolo/Deduplication/duplicates.csv")